Trying to compare Yandex Vision and Face++ resoults
What left to do:
* Found how to save and read label with count of faces
* Count some data from the last block
* Try to make some plots
* Visualize it in streamlit
* Write beutiful README and push it to github

In [47]:
import base64
import json
import requests
import os

Constants:

In [48]:

YANDEX_URL = 'https://vision.api.cloud.yandex.net/vision/v1/batchAnalyze'
FACE_URL = 'https://api-us.faceplusplus.com/facepp/v3/detect'


with open('../yandex_tkn.txt', 'r', encoding='utf-8') as f:
    YANDEX_API_KEY = f.read()

with open('../faceplus_tkn.txt', 'r', encoding='utf-8') as f:
    FACE_API_KEY = f.read()

with open('../faceplus_secret.txt', 'r', encoding='utf-8') as f:
    FACE_API_SECRET = f.read()


YANDEX_HEADER = {"Authorization": f"Api-Key {YANDEX_API_KEY}"}

#faceplusplus requierse to send api key and secret with the image
#so i'll make it later

In this variables i'll keep count of detected faces

In [49]:
yandex_total_faces_count = 0
faceplus_total_faces_count = 0
total_faces_count = 0


#accurasy of every predict in procents
yandex_individual_accurasy = []
faceplus_individual_accurasy = []

Functions:

In [50]:
#taking image file in base64 encoding
#return count of people on the image
def yandex_request(image_b64):
    
    data_dict={"folderId":"b1gb2477drh0ivbugf0p", "analyze_specs": [{"content":image_b64.decode(), 
                "features":[{"type": "FACE_DETECTION"}], "mimeType":"image/jpeg"}]}
    
    response = requests.post(YANDEX_URL, json=data_dict, headers=YANDEX_HEADER)

    responce_data = response.json()

    #if no faces detected return  0
    if(responce_data['results'][0]['results'][0]['faceDetection'] == {}):   return 0

    #array of dicts(coordinates of bounding boxes)
    faces = responce_data['results'][0]['results'][0]['faceDetection']['faces']
    

    return len(faces)

In [51]:
#taking image file in base64 encoding
#return count of people on the image
def faceplus_request(image_b64):

    data_dict = {'api_key':FACE_API_KEY, 'api_secret':FACE_API_SECRET, "image_base64":image_b64}

    response = requests.post(FACE_URL, data=data_dict)

    response_data = json.loads(response.text)

    return response_data['face_num']


Getting data from services:

In [52]:

for filename in os.listdir('data'):
    with open(f'data/{filename}', 'rb') as img:

        image_b64 = base64.b64encode(img.read())
        
        #yandex detecting:
        faces_count = yandex_request(image_b64)
        print (f'Yndx: {faces_count} faces on {filename}')
        yandex_total_faces_count += faces_count

        #face++ detecting:
        faces_count = faceplus_request(image_b64)
        print(f'F++: {faces_count} faces on {filename}')
        faceplus_total_faces_count += faces_count

print(  f'Yandex detected {yandex_total_faces_count} faces', 
        f'Face++ detected {faceplus_total_faces_count} face', 
        f'There\'re {TOTAL_FACES_COUNT} faces at all',
    sep='\n')

Yndx: 0 faces on 144639305.jpg
F++: 21 faces on 144639305.jpg
Yndx: 3 faces on brigade.jpg
F++: 4 faces on brigade.jpg
Yndx: 1 faces on day_of_death.jpg
F++: 2 faces on day_of_death.jpg
Yndx: 3 faces on film.jpg
F++: 3 faces on film.jpg
Yndx: 0 faces on group.jpg
F++: 13 faces on group.jpg
Yndx: 0 faces on live.jpg
F++: 16 faces on live.jpg
Yndx: 1 faces on man.jpg
F++: 1 faces on man.jpg
Yndx: 1 faces on man_in_facemask.jpg
F++: 1 faces on man_in_facemask.jpg
Yndx: 1 faces on me.jpg
F++: 2 faces on me.jpg
Yndx: 5 faces on meme.jpg
F++: 6 faces on meme.jpg
Yndx: 0 faces on people-different-ages-demos-ss-1920.jpg
F++: 13 faces on people-different-ages-demos-ss-1920.jpg
Yndx: 0 faces on people.jpg
F++: 14 faces on people.jpg
Yndx: 5 faces on tranfer-resource-page.jpg
F++: 5 faces on tranfer-resource-page.jpg
Yandex detected 20 faces
Face++ detected 101 face
There're 0 faces at all


Analysing data:
* Total accuracy
* Individual simple avarege mean
* Individual simple avarege weighted mean

In [53]:
yandex_total_accuracy = yandex_total_faces_count / total_faces_count * 100
faceplus_total_accuracy = faceplus_total_faces_count / total_faces_count * 100

print(f'Yndx total accurasy = {yandex_total_accuracy}%', 
    f'Face++ total accurasy = {faceplus_individual_accurasy}%', sep='\n')



ZeroDivisionError: division by zero